# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

Ensure custom resource validation is activated

In [68]:
!minikube start --feature-gates=CustomResourceValidation=true

Starting local Kubernetes v1.8.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


Install Helm

In [77]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


Label the node to allow load testing to run on it

In [78]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "validation" labeled


## Start seldon-core

In [79]:
!helm install ../../helm-charts/seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set cluster_manager.image.tag=0.3-SNAPSHOT \
    --set apife.image.tag=0.1-SNAPSHOT \
    --set engine.image.tag=0.2-SNAPSHOT

NAME:   seldon-core
LAST DEPLOYED: Wed Dec  6 19:48:11 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <none>         3s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
grafana-prom-import-dashboards-s5zj6      0/1    ContainerCreating  0         3s
alertmanager-deployment-6f5c9c5f58-7xmkv  0/1    ContainerCreating  0         3s
seldon-apiserver-69cdfcb4bf-jnb4s         0/1    ContainerCreating  0         3s
seldon-cluster-manager-59fc6c966f-bppqf   0/1    ContainerCreating  0         3s
grafana-prom-deployment-bf46c5988-6g2bl   0/1    ContainerCreating  0         3s
kafka-68cc697c5-l4788                     0/1    Pending            0         2s
prometheus-node-exporter-44th9            0/1    ContainerCreating  0         2s
prometheus-depl

In [66]:
!helm status seldon-core

LAST DEPLOYED: Wed Dec  6 10:46:16 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                           TYPE    DATA  AGE
cluster-manager-client-secret  Opaque  1     11s
grafana-prom-secret            Opaque  1     11s

==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     11s
grafana-import-dashboards  5     11s
prometheus-rules           4     11s
prometheus-server-conf     1     11s

==> v1/Pod
NAME         READY  STATUS   RESTARTS  AGE
zookeeper-1  0/1    Pending  0         11s
zookeeper-2  0/1    Pending  0         11s
zookeeper-3  0/1    Pending  0         11s

==> v1/Pod(related)
grafana-prom-import-dashboards-kx25g      1/1  Running            0  11s
alertmanager-deployment-6f5c9c5f58-5rb55  0/1  ContainerCreating  0  11s
seldon-apiserver-69cdfcb4bf-zdshw         0/1  ContainerCreating  0  11s
seldon-cluster-manager-59fc6c966f-cnk8m   0/1  ContainerCreating  0  11s
grafana-prom-deployment-bf46c5988-tm6d7   0/1  Cont

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [80]:
!kubectl create -f resources/model_invalid1.json

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"apiVersion":"machinelearning.seldon.io/v1alpha1", "kind":"SeldonDeployment", "metadata":map[string]interface {}{"clusterName":"", "namespace":"default", "deletionTimestamp":interface {}(nil), "deletionGracePeriodSeconds":(*int64)(nil), "uid":"cfc66f28-dabe-11e7-adc9-080027a3a01a", "selfLink":"", "initializers":interface {}(nil), "labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "creationTimestamp":"2017-12-06T19:51:12Z"}, "spec":map[string]interface {}{"annotations":map[string]interface {}{"deployment_version":"v1", "project_name":"FX Market Prediction"}, "name":"test-deployment", "oauth_key":1234, "oauth_secret":"oauth-secret", "predictors":[]interface {}{map[string]interface {}{"annotations":map[string]interface {}{"predictor_version":"v1"}, "componentSpec":map[string]interface {}{"spec":map[string]interface {}{"containers":[]interface {}{map[st

For the more complex cases, e.g. checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [81]:
!kubectl create -f resources/model_invalid2.json

seldondeployment "seldon-deployment-example" created


In [82]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[description:Can't find container for predictive unit with name mean-classifier_ state:FAILED]

In [83]:
!kubectl delete -f resources/model_invalid2.json

seldondeployment "seldon-deployment-example" deleted


## Normal Operation

In [84]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
       

## Create Seldon Deployment

In [85]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [86]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   0s


In [87]:
!kubectl describe seldondeployments seldon-deployment-example 

Name:         seldon-deployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"default","self...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2017-12-06T19:52:01Z
  Generation:          0
  Initializers:        <nil>
  Resource Version:    3942
  Self Link:           /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/seldondeployments/seldon-deployment-example
  UID:                 ed32c33e-dabe-11e7-adc9-080027a3a01a
Spec:
  Annotations:
    Deployment _ Version:  v1
    Project _ Name:        FX Market Prediction
  Endpoint:
    Service _ Host:  test-deployment
    Service _ Port:  8000
    Type:            REST
  Name:              test-deployment
  Oauth _ Key:       oauth-key


In [89]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1]]]

## Get predictions

In [90]:
%%bash
SERVER=192.168.99.100:30032
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"data":{"names":["a","b"],"tensor":{"shape":[1,2],"values":[0,0]}}}'

{
  "meta": {
    "puid": "b189c5r53idkq4lk26ekmh98qf",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.05133579311531625]
    }
  }
}

## Update deployment with canary

In [91]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
         

In [92]:
!kubectl apply -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" configured


In [94]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1] map[name:test-deployment-fx-market-predictor-canary replicas:1 replicasAvailable:1]]]

In [95]:
%%bash
SERVER=192.168.99.100:30032
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"data":{"names":["a","b"],"tensor":{"shape":[1,2],"values":[0,0]}}}'

{
  "meta": {
    "puid": "1dsmpd8q98s85phqtf47rbp4hn",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.05133579311531625]
    }
  }
}

## Load test

In [98]:
!helm install ../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret 

NAME:   loadtest
LAST DEPLOYED: Wed Dec  6 20:51:59 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP     EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.98.156.223  <none>       5557:31903/TCP,5558:31975/TCP,8089:31839/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-pg47v   0/1    ContainerCreating  0         0s
locust-master-1-dxrcp  0/1    ContainerCreating  0         0s
locust-master-1-pzhzj  0/1    Terminating        0         55m




# Tear down

In [99]:
!helm delete loadtest --purge

release "loadtest" deleted


In [100]:
!kubectl delete -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" deleted


In [101]:
!helm delete seldon-core --purge

release "seldon-core" deleted
